In [1]:
!pip install pulp #Python Linear Programming
import pulp
!apt-get install -y -qq glpk-utils
from pulp import GLPK
import pandas as pd
import numpy as np

from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/Colab Notebooks/Modelling")

df = pd.read_csv("Modelling data.csv")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 27.7 MB/s eta 0:00:00
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 120882 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_5.0-1_amd64.deb ...
Unpackin

In [38]:
weights_matrix = np.empty((9,9))
for i in range(9):
    model = pulp.LpProblem("Investment Analysis", pulp.LpMaximize) # Create an LP maximization problem
    m1 = pulp.LpVariable("m1", lowBound=0, upBound=None, cat='Continuous') # Create a variable m1>=0
    m2 = pulp.LpVariable("m2", lowBound=0, upBound=None, cat='Continuous') # Create a variable m2>=0
    n1 = pulp.LpVariable("n1", lowBound=0, upBound=None, cat='Continuous') # Create a variable n1>=0
    n2 = pulp.LpVariable("n2", lowBound=0, upBound=None, cat='Continuous') # Create a variable n2>=0



    model += m1*df["Y1"][i] + m2*df["Y2"][i]  #objective function

    model += n1*df["X1"][i] + n2*df["X2"][i] == 1 #first constraint (equality)

    for j in range(9):
        model += m1*df["Y1"][j] + m2*df["Y2"][j] - n1*df["X1"][j] - n2*df["X2"][j] <= 0

    status = model.solve(GLPK(msg=True, options=['--ranges', 'sensitivity.txt']))
    print("Model Status:{}".format(pulp.LpStatus[model.status]))

    list_weights=[]
    for k in model.variables():
	      print(k.name, "=", k.varValue, "\tReduced Cost =", k.dj)
    for k in model.variables():
        list_weights=list_weights+[k.varValue]
    for l in range(4):
        value= list_weights[l]
        weights_matrix[l,i] = value

    print("Objective=", pulp.value(model.objective))

    # Displaying Shadow Price & Slack Value per Constraint
    print ("\nSensitivity Analysis\nConstraint\t\tShadow Price\tSlack")
    for name, m in model.constraints.items():
	      print(name, ":", m, "\t", m.pi, "\t\t", m.slack)

Model Status:Optimal
m1 = 0.0 	Reduced Cost = None
m2 = 2.61 	Reduced Cost = None
n1 = 0.0 	Reduced Cost = None
n2 = 1.33333 	Reduced Cost = None
Objective= 0.9999999988199999

Sensitivity Analysis
Constraint		Shadow Price	Slack
_C1 : 114.8*n1 + 0.75*n2 = 1.0 	 None 		 None
_C2 : 8.71*m1 + 0.383141762*m2 - 114.8*n1 - 0.75*n2 <= -0.0 	 None 		 None
_C3 : 8.1*m1 + 0.01618123*m2 - 106.8*n1 - 0.7*n2 <= -0.0 	 None 		 None
_C4 : 6.2*m1 + 0.041407867*m2 - 84.6*n1 - 0.86*n2 <= -0.0 	 None 		 None
_C5 : 3.12*m1 + 0.031152648*m2 - 77.7*n1 - 0.63*n2 <= -0.0 	 None 		 None
_C6 : 0.12*m1 + 0.071428571*m2 - 56.3*n1 - 0.81*n2 <= -0.0 	 None 		 None
_C7 : 1.32*m1 + 0.043956044*m2 - 45.8*n1 - 0.73*n2 <= -0.0 	 None 		 None
_C8 : 3.07*m1 + 0.1953125*m2 - 37.8*n1 - 0.76*n2 <= -0.0 	 None 		 None
_C9 : 6.28*m1 + 0.0253614*m2 - 36.7*n1 - 0.71*n2 <= -0.0 	 None 		 None
_C10 : 1.88*m1 + 0.031786395*m2 - 29.7*n1 - 0.68*n2 <= -0.0 	 None 		 None
Model Status:Optimal
m1 = 0.123155 	Reduced Cost = None
m2 = 0.0

In [40]:
efficiency_table =np.empty((9,9))
average_list =[]
for row in range(9):
    row_values=[]
    for column in range(9):
      value = ((weights_matrix[0,column]*df["Y1"][row])+(weights_matrix[1,column]*df["Y2"][row]))/((weights_matrix[2,column]*df["X1"][row])+(weights_matrix[3,column]*df["X2"][row]))

      efficiency_table[row,column]=value
      row_values=row_values+[value]
    from statistics import mean
    average =mean(row_values)
    average_list =average_list+[average]

In [48]:
average_list_rounded = [ '%.2f' % elem for elem in average_list ]

In [49]:
country_list = ['Brazil','Vietnam','Chile','India','Kazakhstan','Egypt','Mexico','Indonesia','Morocco' ]

In [50]:
for country in country_list:
    index = country_list.index(country)
    print("Cross-efficiency score for",country,"=",average_list_rounded[index])

Cross-efficiency score for Brazil = 0.97
Cross-efficiency score for Vietnam = 0.57
Cross-efficiency score for Chile = 0.46
Cross-efficiency score for India = 0.30
Cross-efficiency score for Kazakhstan = 0.13
Cross-efficiency score for Egypt = 0.21
Cross-efficiency score for Mexico = 0.70
Cross-efficiency score for Indonesia = 0.70
Cross-efficiency score for Morocco = 0.29


In [44]:
display(efficiency_table)
np.set_printoptions(suppress=True, precision = 2)
print(efficiency_table)
np.savetxt('table.txt', efficiency_table, fmt='%f')

array([[1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 0.77],
       [0.05, 1.  , 1.  , 1.  , 0.05, 0.55, 0.05, 1.  , 0.43],
       [0.09, 0.71, 0.71, 0.71, 0.13, 0.53, 0.13, 0.71, 0.45],
       [0.1 , 0.46, 0.46, 0.46, 0.11, 0.32, 0.11, 0.46, 0.26],
       [0.17, 0.02, 0.02, 0.02, 0.29, 0.17, 0.29, 0.02, 0.15],
       [0.12, 0.2 , 0.2 , 0.2 , 0.21, 0.28, 0.21, 0.2 , 0.26],
       [0.5 , 0.46, 0.46, 0.46, 1.  , 1.  , 1.  , 0.46, 1.  ],
       [0.07, 1.  , 1.  , 1.  , 0.14, 1.  , 0.14, 1.  , 1.  ],
       [0.09, 0.32, 0.32, 0.32, 0.19, 0.43, 0.19, 0.32, 0.46]])

[[1.   1.   1.   1.   1.   1.   1.   1.   0.77]
 [0.05 1.   1.   1.   0.05 0.55 0.05 1.   0.43]
 [0.09 0.71 0.71 0.71 0.13 0.53 0.13 0.71 0.45]
 [0.1  0.46 0.46 0.46 0.11 0.32 0.11 0.46 0.26]
 [0.17 0.02 0.02 0.02 0.29 0.17 0.29 0.02 0.15]
 [0.12 0.2  0.2  0.2  0.21 0.28 0.21 0.2  0.26]
 [0.5  0.46 0.46 0.46 1.   1.   1.   0.46 1.  ]
 [0.07 1.   1.   1.   0.14 1.   0.14 1.   1.  ]
 [0.09 0.32 0.32 0.32 0.19 0.43 0.19 0.32 0.46]]
